# <span style="color:white; font-family:PT Sans Narrow; font-size:1.1em"> Import Packages </span>

In [1]:
%reload_ext autoreload
%autoreload 2

import os, sys 
data_dir = os.path.dirname(os.path.realpath('.'))
sys.path.append(data_dir)


from crowdkit.datasets import load_dataset
import pandas as pd
import ipywidgets
import load_data 


print('data_dir:', data_dir)

%reload_ext load_data

2021-12-14 03:00:08.484960: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-14 03:00:08.485003: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


data_dir: /workspace/crowd-kit


# <span style="color:white; opacity:0.7; font-family:PT Sans Narrow; font-size:1.3em"> 1. Loading the Data </span>
## <span style="color:green; font-family:PT Sans Narrow; font-size:1.1em"> 1.1 Selecting the Dataset </span>

In [2]:
dataset = ipywidgets.Dropdown( options = [ ('1. kr-vs-kp'    ,'kr-vs-kp'), 
                                           ('2. mushroom'    ,'mushroom'),
                                           ('3. sick'        ,'sick'),
                                           ('4. spambase'    ,'spambase'),
                                           ('5. tic-tac-toe' ,'tic-tac-toe'),
                                           ('6. splice'      ,'splice'),
                                           ('8. waveform'    ,'waveform'),
                                           ('9. biodeg'      ,'biodeg'),
                                           ('10. horse-colic','horse-colic'),
                                           ('11. ionosphere' ,'ionosphere'),
                                           ('12. vote'       ,'vote')],      
                                value = 'vote')


@ipywidgets.interact(WHICH_DATASET = dataset)
def read_data(WHICH_DATASET):
    if WHICH_DATASET in ['sick','splice','biodeg','vote','horse-colic']:
        print('dataset does not exist')

    else:
        
        data, feature_columns = load_data.aim1_3_read_download_UCI_database(WHICH_DATASET=WHICH_DATASET, mode='read')
        print(data['train'].head(3))
        print('train shape:',data['train'].shape)

interactive(children=(Dropdown(description='WHICH_DATASET', index=10, options=(('1. kr-vs-kp', 'kr-vs-kp'), ('…

In [4]:
data

NameError: name 'data' is not defined

## <span style="color:Green; font-family:PT Sans Narrow; font-size:1.1em"> 1.1 Preprocessing the data to adapt to this package </span>

In [4]:
datasets_list = ['TlkAgg-Categorical']
data_mode = 'synthetic' # 'synthetic' or 'real'

df, df_gt = load_dataset('relevance-2')

In [5]:
df

,performer,task,label
0,w851,t30685,1
1,w6991,t30008,0
2,w2596,t36316,0
3,w5507,t15145,1
4,w2982,t44785,1
...,...,...,...
475531,w4660,t62250,1
475532,w6630,t46626,0
475533,w4605,t93513,1
475534,w1928,t29002,0


In [6]:
feature_columns

NameError: name 'feature_columns' is not defined

In [ ]:
df[df.task=='t49052'].performer.shape

(5,)